In [164]:
import pandas as pd

# leer dataset 
samples_nans = pd.read_csv('../data/samples_nans.csv', sep=',')
samples_nans['date_time_init'] = pd.to_datetime(samples_nans['date_time_init'])
samples_nans['date_time_end'] = pd.to_datetime(samples_nans['date_time_end'])
samples_nans

,id_sample,id_link,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
0,0,0,100,inicio,101,primer stop,100:101,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,0,1,101,primer stop,102,segundo stop,101:102,2023-03-01 07:27:33,2023-03-01 07:29:19,106.0,13190622.0,13190922.0,SJ01,1.0,2023-03-01
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-03-01 07:29:19,2023-03-01 07:30:49,90.0,13190922.0,13191197.0,SJ01,1.0,2023-03-01
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-03-01 07:30:49,2023-03-01 07:31:13,24.0,13191197.0,13191276.0,SJ01,1.0,2023-03-01
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-03-01 07:31:13,2023-03-01 07:32:29,76.0,13191276.0,13191535.0,SJ01,1.0,2023-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278958,3360,78,178,Primero,179,Segundo,178:179,2023-03-25 11:08:40,2023-03-25 11:08:52,12.0,17265203.0,17265239.0,SJ42,2.0,2023-03-25
278959,3360,79,179,Segundo,180,Reservorio,179:180,2023-03-25 11:08:52,2023-03-25 11:09:16,24.0,17265239.0,17265310.0,SJ42,2.0,2023-03-25
278960,3360,80,180,Reservorio,181,Cuarto,180:181,2023-03-25 11:09:16,2023-03-25 11:09:28,12.0,17265310.0,17265340.0,SJ42,2.0,2023-03-25
278961,3360,81,181,Cuarto,182,Quinto,181:182,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
import numpy as np

can_nans = samples_nans['time_travel'].isna().sum()
print(f"{can_nans} nans de {samples_nans.shape[0]} rows equivale {can_nans*100/samples_nans.shape[0]:0.0f}%")

41122 nans de 278963 rows equivale 15%


### Reemplazar NaNs con promedios

In [166]:
from more_itertools import consecutive_groups

def intervals(numeros):
    intervals = []
    for group in consecutive_groups(numeros):
        group = list(group)
        if len(group) == 1:
            intervals.append(tuple(group))
        else:
            intervals.append(tuple(group))

    return intervals

In [167]:
from datetime import datetime, timedelta

means_dict = samples_nans.groupby('link', observed=False).agg({'time_travel':[np.mean]})[('time_travel', 'mean')].to_list()

list_df_listos = [] 
for id, df_sample in samples_nans.groupby('id_sample'):
    # identificar los intervalos con nans
    df_sample.reset_index(inplace=True)
    index_nans = df_sample.index[pd.isna(df_sample['time_travel'])]
    inter_nans = intervals(list(index_nans))
    df_sample.insert(loc=2, column='id_linkref', value=df_sample.index)

    for gap in inter_nans:
        # generar los time travel
        means = [means_dict[i] for i in gap]

        # si hay nans en el medio, ajustamos means
        if gap[0] != 0 and gap[-1] != 82:
            time_init = df_sample.loc[gap[0] - 1, 'date_time_end']
            time_end = df_sample.loc[gap[-1] + 1, 'date_time_init']
            dif_time =  time_end - time_init
            means = list(map(lambda e: dif_time.total_seconds() * (e / sum(means)), means))

        # colocamos means
        for i in range(len(gap)):
            df_sample.loc[gap[i], 'time_travel'] = round(means[i], 0)
            if 0 not in gap:
                df_sample.loc[gap[i], 'date_time_init'] = df_sample.loc[gap[i]-1, 'date_time_end']
                df_sample.loc[gap[i], 'date_time_end'] = df_sample.loc[gap[i], 'date_time_init'] + timedelta(seconds=round(means[i], 0))
            else:
                j = (i+1)*(-1)
                df_sample.loc[gap[j], 'date_time_end'] = df_sample.loc[gap[j]+1, 'date_time_init']
                df_sample.loc[gap[j], 'date_time_init'] = df_sample.loc[gap[j], 'date_time_end'] - timedelta(seconds=round(means[i], 0))

    df_sample.drop(columns=['index', 'id_link'], inplace=True)                
    list_df_listos.append(df_sample)


In [168]:
# Extraer los samples con tiempo que sobrepasan el min y max permitido
min = 10
max = 1500
list_err = []
listo = []
samplesdf = pd.concat(list_df_listos)
for id, sampledf in samplesdf.groupby('id_sample'):
    min_sample = sampledf['time_travel'].min()
    max_sample = sampledf['time_travel'].max()
    if (min_sample < min) or (max_sample > max):
        list_err.append(sampledf)
    else:
        listo.append(sampledf)

# Mostrar
print(f"{len(listo)} samples correctos")
print(f"{len(list_err)} samples con max, min trave times erroneos")

2992 samples correctos
369 samples con max, min trave times erroneos


In [169]:
samplesdf

,id_sample,id_linkref,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date
0,0,0,100,inicio,101,primer stop,100:101,2023-03-01 07:24:21,2023-03-01 07:27:33,192.0,NaN,NaN,NaN,NaN,NaN
1,0,1,101,primer stop,102,segundo stop,101:102,2023-03-01 07:27:33,2023-03-01 07:29:19,106.0,13190622.0,13190922.0,SJ01,1.0,2023-03-01
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-03-01 07:29:19,2023-03-01 07:30:49,90.0,13190922.0,13191197.0,SJ01,1.0,2023-03-01
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-03-01 07:30:49,2023-03-01 07:31:13,24.0,13191197.0,13191276.0,SJ01,1.0,2023-03-01
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-03-01 07:31:13,2023-03-01 07:32:29,76.0,13191276.0,13191535.0,SJ01,1.0,2023-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,3360,78,178,Primero,179,Segundo,178:179,2023-03-25 11:08:40,2023-03-25 11:08:52,12.0,17265203.0,17265239.0,SJ42,2.0,2023-03-25
79,3360,79,179,Segundo,180,Reservorio,179:180,2023-03-25 11:08:52,2023-03-25 11:09:16,24.0,17265239.0,17265310.0,SJ42,2.0,2023-03-25
80,3360,80,180,Reservorio,181,Cuarto,180:181,2023-03-25 11:09:16,2023-03-25 11:09:28,12.0,17265310.0,17265340.0,SJ42,2.0,2023-03-25
81,3360,81,181,Cuarto,182,Quinto,181:182,2023-03-25 11:09:28,2023-03-25 11:09:43,15.0,NaN,NaN,NaN,NaN,NaN


In [170]:
# verificar maximos y minimos
data = pd.concat(listo).copy().reset_index(drop=True)
print('Minimos:')
print(data.nsmallest(10, 'time_travel')['time_travel'].to_string())
print('='*20)
print('Maximos:')
print(data.nlargest(10, 'time_travel')['time_travel'].to_string())

Minimos:
410     10.0
757     10.0
770     10.0
5224    10.0
6545    10.0
6650    10.0
6877    10.0
7292    10.0
8544    10.0
9125    10.0
Maximos:
69222     1493.0
209077    1492.0
161020    1457.0
43326     1431.0
104331    1369.0
197043    1295.0
9960      1290.0
148653    1251.0
196959    1204.0
108813    1197.0


### Poner intervalos de tiempo

In [171]:
import pandas as pd
from collections import Counter
from datetime import datetime, time, timedelta

# Convertir las columnas de tiempo en datetime
data['date_time_init'] = pd.to_datetime(data['date_time_init'])
data['date_time_end'] = pd.to_datetime(data['date_time_end'])

# aproximar a horas las datetime (para hacer el histograma)
ix = pd.DatetimeIndex(data['date_time_end']).floor('1h')
data['time'] = ix
data['time'] = data['time'].apply(lambda dt: str(dt).split()[-1]) #

# Extraer la hora en otro columna
data['hour'] = data['date_time_end'].apply(lambda dt: dt.time().strftime("%H:%M:%S"))
data.head()

,id_sample,id_linkref,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date,time,hour
0,0,0,100,inicio,101,primer stop,100:101,2023-03-01 07:24:21,2023-03-01 07:27:33,192.0,NaN,NaN,NaN,NaN,NaN,07:00:00,07:27:33
1,0,1,101,primer stop,102,segundo stop,101:102,2023-03-01 07:27:33,2023-03-01 07:29:19,106.0,13190622.0,13190922.0,SJ01,1.0,2023-03-01,07:00:00,07:29:19
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-03-01 07:29:19,2023-03-01 07:30:49,90.0,13190922.0,13191197.0,SJ01,1.0,2023-03-01,07:00:00,07:30:49
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-03-01 07:30:49,2023-03-01 07:31:13,24.0,13191197.0,13191276.0,SJ01,1.0,2023-03-01,07:00:00,07:31:13
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-03-01 07:31:13,2023-03-01 07:32:29,76.0,13191276.0,13191535.0,SJ01,1.0,2023-03-01,07:00:00,07:32:29


In [172]:
def redondear_hora(dt, split):
    m = int(round(dt.time().minute/split, 0))*split
    if m == 60 or dt.time().hour == 4: 
        return time(dt.time().hour + 1, 0)   
    else:
        return time(dt.time().hour, m)   

#  Aproximar la 'hora' en intervalos de tiempo n minutos
n = 15
name_col = 'rg' + str(n)+'min'
data[name_col] = data['date_time_end'].apply(redondear_hora, split=n)
data.head(85)

,id_sample,id_linkref,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date,time,hour,rg15min
0,0,0,100,inicio,101,primer stop,100:101,2023-03-01 07:24:21,2023-03-01 07:27:33,192.0,NaN,NaN,NaN,NaN,NaN,07:00:00,07:27:33,07:30:00
1,0,1,101,primer stop,102,segundo stop,101:102,2023-03-01 07:27:33,2023-03-01 07:29:19,106.0,13190622.0,13190922.0,SJ01,1.0,2023-03-01,07:00:00,07:29:19,07:30:00
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-03-01 07:29:19,2023-03-01 07:30:49,90.0,13190922.0,13191197.0,SJ01,1.0,2023-03-01,07:00:00,07:30:49,07:30:00
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-03-01 07:30:49,2023-03-01 07:31:13,24.0,13191197.0,13191276.0,SJ01,1.0,2023-03-01,07:00:00,07:31:13,07:30:00
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-03-01 07:31:13,2023-03-01 07:32:29,76.0,13191276.0,13191535.0,SJ01,1.0,2023-03-01,07:00:00,07:32:29,07:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0,80,180,Reservorio,181,Cuarto,180:181,2023-03-01 08:39:35,2023-03-01 08:39:57,22.0,13204848.0,13204933.0,SJ01,1.0,2023-03-01,08:00:00,08:39:57,08:45:00
81,0,81,181,Cuarto,182,Quinto,181:182,2023-03-01 08:39:57,2023-03-01 08:40:11,14.0,13204933.0,13204978.0,SJ01,1.0,2023-03-01,08:00:00,08:40:11,08:45:00
82,0,82,182,Quinto,183,Sexto,182:183,2023-03-01 08:40:11,2023-03-01 08:41:09,58.0,13204978.0,13205166.0,SJ01,1.0,2023-03-01,08:00:00,08:41:09,08:45:00
83,1,0,100,inicio,101,primer stop,100:101,2023-03-01 10:34:33,2023-03-01 10:35:55,82.0,13228633.0,13228905.0,SJ01,2.0,2023-03-01,10:00:00,10:35:55,10:30:00


In [173]:
# dividir las horas y minutos en diferentes columnas
data['horas'] = data['rg15min'].apply(lambda t: t.hour)
data['minute'] = data['rg15min'].apply(lambda t: t.minute)
data

,id_sample,id_linkref,init_stop,init_name,end_stop,end_name,link,date_time_init,date_time_end,time_travel,init_idp,end_idp,veh,lap,date,time,hour,rg15min,horas,minute
0,0,0,100,inicio,101,primer stop,100:101,2023-03-01 07:24:21,2023-03-01 07:27:33,192.0,NaN,NaN,NaN,NaN,NaN,07:00:00,07:27:33,07:30:00,7,30
1,0,1,101,primer stop,102,segundo stop,101:102,2023-03-01 07:27:33,2023-03-01 07:29:19,106.0,13190622.0,13190922.0,SJ01,1.0,2023-03-01,07:00:00,07:29:19,07:30:00,7,30
2,0,2,102,segundo stop,103,Ccollana,102:103,2023-03-01 07:29:19,2023-03-01 07:30:49,90.0,13190922.0,13191197.0,SJ01,1.0,2023-03-01,07:00:00,07:30:49,07:30:00,7,30
3,0,3,103,Ccollana,104,Puente Huaccoto,103:104,2023-03-01 07:30:49,2023-03-01 07:31:13,24.0,13191197.0,13191276.0,SJ01,1.0,2023-03-01,07:00:00,07:31:13,07:30:00,7,30
4,0,4,104,Puente Huaccoto,105,Kayra,104:105,2023-03-01 07:31:13,2023-03-01 07:32:29,76.0,13191276.0,13191535.0,SJ01,1.0,2023-03-01,07:00:00,07:32:29,07:30:00,7,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248331,3360,78,178,Primero,179,Segundo,178:179,2023-03-25 11:08:40,2023-03-25 11:08:52,12.0,17265203.0,17265239.0,SJ42,2.0,2023-03-25,11:00:00,11:08:52,11:15:00,11,15
248332,3360,79,179,Segundo,180,Reservorio,179:180,2023-03-25 11:08:52,2023-03-25 11:09:16,24.0,17265239.0,17265310.0,SJ42,2.0,2023-03-25,11:00:00,11:09:16,11:15:00,11,15
248333,3360,80,180,Reservorio,181,Cuarto,180:181,2023-03-25 11:09:16,2023-03-25 11:09:28,12.0,17265310.0,17265340.0,SJ42,2.0,2023-03-25,11:00:00,11:09:28,11:15:00,11,15
248334,3360,81,181,Cuarto,182,Quinto,181:182,2023-03-25 11:09:28,2023-03-25 11:09:43,15.0,NaN,NaN,NaN,NaN,NaN,11:00:00,11:09:43,11:15:00,11,15


In [174]:
# guardar datos listos para entrenar
data.to_csv("../data/travel_time_series.csv", index=False)